# Bank Churn Prediction Proyect

## Introduction

En este proyecto vamos a analizar algunos datos del banco "Beta Bank" ya que los clientes se estan saliendo del banco poco a poco, asi que nuestro trabajo sera predecir si alguno de 
los clientes actuales dejara el banco pronto para asi poder tomar medidas y evitar que esto suceda. Crearemos un modelo con el maximo F1 posible.



## Objectives

Desarrollar un modelo predictivo eficiente que identifique con alta precisión a los clientes con mayor riesgo de abandonar los servicios de Beta Bank. Este modelo ayudará a la entidad bancaria a implementar estrategias proactivas para la retención de clientes, optimizando recursos y mejorando la satisfacción del cliente.

## Importacion de datos

In [27]:
# Manipulación y análisis de datos
import pandas as pd
import numpy as np

# Preprocesamiento de datos
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.utils import shuffle


# Modelos de machine learning
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score, recall_score, f1_score, roc_auc_score

## Carga de data sets

In [28]:
df = pd.read_csv('datasets/Churn.csv')

## Analisis y preparacion de datos

### Exploracion Inicial

In [29]:
df.head()

,RowNumber,CustomerId,Surname,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,1,15634602,Hargrave,619,France,Female,42,2.0,0.00,1,1,1,101348.88,1
1,2,15647311,Hill,608,Spain,Female,41,1.0,83807.86,1,0,1,112542.58,0
2,3,15619304,Onio,502,France,Female,42,8.0,159660.80,3,1,0,113931.57,1
3,4,15701354,Boni,699,France,Female,39,1.0,0.00,2,0,0,93826.63,0
4,5,15737888,Mitchell,850,Spain,Female,43,2.0,125510.82,1,1,1,79084.10,0


Características
- RowNumber: índice de cadena de datos
- CustomerId: identificador de cliente único
- Surname: apellido
- CreditScore: valor de crédito
- Geography: país de residencia
- Gender: sexo
- Age: edad
- Tenure: período durante el cual ha madurado el depósito a plazo fijo de un cliente (años)
- Balance: saldo de la cuenta
- NumOfProducts: número de productos bancarios utilizados por el cliente
- HasCrCard: el cliente tiene una tarjeta de crédito (1 - sí; 0 - no)
- IsActiveMember: actividad del cliente (1 - sí; 0 - no)
- EstimatedSalary: salario estimado

Objetivo
- Exited: El cliente se ha ido (1 - sí; 0 - no)

In [30]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10000 entries, 0 to 9999
Data columns (total 14 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   RowNumber        10000 non-null  int64  
 1   CustomerId       10000 non-null  int64  
 2   Surname          10000 non-null  object 
 3   CreditScore      10000 non-null  int64  
 4   Geography        10000 non-null  object 
 5   Gender           10000 non-null  object 
 6   Age              10000 non-null  int64  
 7   Tenure           9091 non-null   float64
 8   Balance          10000 non-null  float64
 9   NumOfProducts    10000 non-null  int64  
 10  HasCrCard        10000 non-null  int64  
 11  IsActiveMember   10000 non-null  int64  
 12  EstimatedSalary  10000 non-null  float64
 13  Exited           10000 non-null  int64  
dtypes: float64(3), int64(8), object(3)
memory usage: 1.1+ MB


Podemos ver tenemos algunos datos nulos en Tenure, exactamente 909 los cuales representan un 9% de los datos, no es un porcentaje muy alto, pero si lo suficiente como para tener que tomar una decision sobre ellos ya que no podemos dejarlos asi, ya que podrian afectar a nuestro modelo. 

Tenemos varias opciones las cuales son las siguientes:

- Eliminarlos
- Rellenarlos con la media, mediana o moda
- Rellenarlos con un valor aleatorio

Decidimos eliminarlos ya que no es un porcentaje muy alto y no afectara mucho a nuestro modelo, y asi no tendremos que preocuparnos de ellos.

### Eliminacion de datos nulos

In [31]:
df = df.dropna(subset=['Tenure'])

In [32]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 9091 entries, 0 to 9998
Data columns (total 14 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   RowNumber        9091 non-null   int64  
 1   CustomerId       9091 non-null   int64  
 2   Surname          9091 non-null   object 
 3   CreditScore      9091 non-null   int64  
 4   Geography        9091 non-null   object 
 5   Gender           9091 non-null   object 
 6   Age              9091 non-null   int64  
 7   Tenure           9091 non-null   float64
 8   Balance          9091 non-null   float64
 9   NumOfProducts    9091 non-null   int64  
 10  HasCrCard        9091 non-null   int64  
 11  IsActiveMember   9091 non-null   int64  
 12  EstimatedSalary  9091 non-null   float64
 13  Exited           9091 non-null   int64  
dtypes: float64(3), int64(8), object(3)
memory usage: 1.0+ MB


### Comprobacion de duplicados

In [33]:
print("Cantidad de duplicados =", df.duplicated().sum())


Cantidad de duplicados = 0


Eliminamos los datos nulos y comprobamos que ya no tenemos ninguno.

A demas verificamos si no tenemos valores duplicados, ya que podrian afectar a nuestro modelo.

### Eliminacion de columnas innecesarias

Ya que tenemos columnas que no van a ayudar a nuestro modelo, las vamos a eliminar, estas son RowNumber, Surname y CustomerId.

In [34]:
df = df.drop(columns=['Surname'])
df = df.drop(columns=['RowNumber'])
df = df.drop(columns=['CustomerId'])



## Categorizacion de datos

Ya que tenemos datos categoricos, tenemos que convertirlos a numericos para que nuestro modelo pueda trabajar con ellos. Especificamente Gender y Geography.

In [35]:
df = pd.get_dummies(df, columns=['Geography', 'Gender'])

df.head(5)


,CreditScore,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited,Geography_France,Geography_Germany,Geography_Spain,Gender_Female,Gender_Male
0,619,42,2.0,0.00,1,1,1,101348.88,1,True,False,False,True,False
1,608,41,1.0,83807.86,1,0,1,112542.58,0,False,False,True,True,False
2,502,42,8.0,159660.80,3,1,0,113931.57,1,True,False,False,True,False
3,699,39,1.0,0.00,2,0,0,93826.63,0,True,False,False,True,False
4,850,43,2.0,125510.82,1,1,1,79084.10,0,False,False,True,True,False


## Division del conjunto de Datos

En este paso vamos a dividir nuestro conjunto de datos en 3 partes, train, validacion y test. Para poder entrenar nuestro modelo, validar que funciona correctamente y por ultimo testearlo.

In [36]:
df_train, df_temp = train_test_split(df, test_size=0.2, random_state=12345)
df_val, df_test = train_test_split(df_temp, test_size=0.25, random_state=12345)


## Escalado de Caracteristicas

Dado que algunas de nuestras columnas tienen valores muy altos y otras muy bajos, tenemos que escalarlas para que nuestro modelo pueda trabajar con ellas correctamente. Entre ellas tenemos CreditScore, Age, Tenure, Balance y EstimatedSalary.

In [37]:
scaler = StandardScaler()
num_columns = ['CreditScore', 'Age', 'Tenure', 'Balance', 'NumOfProducts', 'EstimatedSalary']
scaler.fit(df_train[num_columns])


df_train[num_columns] = scaler.transform(df_train[num_columns])
df_val[num_columns] = scaler.transform(df_val[num_columns])
df_test[num_columns] = scaler.transform(df_test[num_columns])

## Verificacion del equilibrio de clases

In [38]:
class_distribution = df_train['Exited'].value_counts(normalize=True)
print(class_distribution)


Exited
0    0.795792
1    0.204208
Name: proportion, dtype: float64


Podemos ver que tenemos un desequilibrio de clases, ya que tenemos un 80% de clientes que no se han ido y un 20% que si se han ido. Esto puede afectar a nuestro modelo, ya que podria aprender a predecir que todos los clientes no se van a ir y tendriamos un modelo que no nos serviria para nada.

Es por eso que tenemos que balancear las clases, para que nuestro modelo aprenda a predecir correctamente.

En este caso vamos a utilizar la tecnica de upsampling, la cual consiste en duplicar los datos de la clase minoritaria.

In [39]:
X_train = df_train.drop('Exited', axis=1)
y_train = df_train['Exited']

minority_class = df_train[df_train['Exited'] == 1]

# Duplicar por 3 la clase minoritaria para o
oversampled_minority_class = pd.concat([minority_class] * 3, ignore_index=True)

# Aleatorizar el orden de las filas (shuffle)
oversampled_minority_class = shuffle(oversampled_minority_class, random_state=12345)

# Combinar las filas duplicadas con las originales
X_train_balanced = pd.concat([X_train, oversampled_minority_class.drop('Exited', axis=1)], ignore_index=True)
y_train_balanced = pd.concat([y_train, oversampled_minority_class['Exited']], ignore_index=True)

Ahora para corroborar vamos a ver el balance de clases de nuevo.

In [40]:
class_distribution_balanced = y_train_balanced.value_counts(normalize=True)
print(class_distribution_balanced)

Exited
1    0.506523
0    0.493477
Name: proportion, dtype: float64


Podemos ver que ahora tenemos un 50% de clientes que se han ido y un 50% que no se han ido, lo cual es perfecto para nuestro modelo.

## Seleccion de Modelos y Entrenamiento

### Seleccion Inicial de Modelos

Para esta parte nos vamos a decidir por 3 modelos, los cuales son los siguientes:

- LogisticRegression
- DecisionTreeClassifier
- RandomForestClassifier

### Logistic Regression

In [41]:
logistic_regression_model = LogisticRegression(random_state=12345, solver='liblinear')

# Entrenar el modelo en el conjunto de entrenamiento
logistic_regression_model.fit(X_train_balanced, y_train_balanced)

# Predecir en el conjunto de validación
y_valid_pred_logistic_regression = logistic_regression_model.predict(df_val.drop('Exited', axis=1))


In [42]:
# Calcular la precisión
accuracy = accuracy_score(df_val['Exited'], y_valid_pred_logistic_regression)

# Calcular el recall
recall = recall_score(df_val['Exited'], y_valid_pred_logistic_regression)

# Calcular el F1-score
f1 = f1_score(df_val['Exited'], y_valid_pred_logistic_regression)

# Calcular el ROC AUC
roc_auc = roc_auc_score(df_val['Exited'], y_valid_pred_logistic_regression)

print(f"Accuracy: {accuracy}")
print(f"Recall: {recall}")
print(f"ROC AUC: {roc_auc}")
print(f"F1 Score: {f1}")

Accuracy: 0.7060117302052786
Recall: 0.7299270072992701
ROC AUC: 0.714963503649635
F1 Score: 0.49937578027465673


Podemos ver que el modelo de regresion logistica nos da un F1 de 0.499, lo cual no es muy bueno, pero es un buen comienzo. Vamos a seguir probando con los otros modelos.

### Decision Tree Classifier

In [43]:
# Seleccionar modelo
decision_tree_model = DecisionTreeClassifier(random_state=12345)

# Entrenar el modelo en el conjunto de entrenamiento
decision_tree_model.fit(X_train_balanced, y_train_balanced)

# Predecir en el conjunto de validación
y_valid_pred_decision_tree = decision_tree_model.predict(df_val.drop('Exited', axis=1))

In [44]:
# Calcular la precisión
accuracy = accuracy_score(df_val['Exited'], y_valid_pred_decision_tree)

# Calcular el recall
recall = recall_score(df_val['Exited'], y_valid_pred_decision_tree)

# Calcular el F1-score
f1 = f1_score(df_val['Exited'], y_valid_pred_decision_tree)

# Calcular el ROC AUC
roc_auc = roc_auc_score(df_val['Exited'], y_valid_pred_decision_tree)

print(f"Accuracy: {accuracy}")
print(f"Recall: {recall}")
print(f"ROC AUC: {roc_auc}")
print(f"F1 Score: {f1}")

Accuracy: 0.7983870967741935
Recall: 0.5182481751824818
ROC AUC: 0.6935277573160115
F1 Score: 0.5080500894454383


Podemos ver que el modelo de arbol de decision nos da un F1 de 0.508, lo cual es un poco mejor que el anterior, pero no es suficiente. Vamos a seguir probando con el ultimo modelo.

### Random Forest Classifier

In [45]:
# Seleccionar modelo
random_forest_model = RandomForestClassifier(random_state=12345)

# Entrenar el modelo en el conjunto de entrenamiento
random_forest_model.fit(X_train_balanced, y_train_balanced)

# Predecir en el conjunto de validación
y_valid_pred_random_forest = random_forest_model.predict(df_val.drop('Exited', axis=1))

In [46]:
# Calcular la precisión
accuracy = accuracy_score(df_val['Exited'], y_valid_pred_random_forest)

# Calcular el recall
recall = recall_score(df_val['Exited'], y_valid_pred_random_forest)

# Calcular el F1-score
f1 = f1_score(df_val['Exited'], y_valid_pred_random_forest)

# Calcular el ROC AUC
roc_auc = roc_auc_score(df_val['Exited'], y_valid_pred_random_forest)

print(f"Accuracy: {accuracy}")
print(f"Recall: {recall}")
print(f"ROC AUC: {roc_auc}")
print(f"F1 Score: {f1}")

Accuracy: 0.8533724340175953
Recall: 0.5401459854014599
ROC AUC: 0.7361280385722896
F1 Score: 0.5967741935483871


Como podemos ver el modelo de random forest nos da un F1 de 0.596, lo cual es el mejor de los 3 modelos que hemos probado.

Es por esto que vamos a indagar mas en este modelo para ver si podemos mejorarlo.

Primero vamos a seleccionar los hiperparametros que mejor funcionan para este modelo.

### Seleccion de Hiperparametros para Random Forest

In [47]:
best_model = None
best_result = 0
best_est = 0
best_depth = 0

for est in range(1, 100, 10):
    for depth in range(1, 20, 2):
        model = RandomForestClassifier(random_state=12345, n_estimators=est, max_depth=depth)
        model.fit(X_train_balanced, y_train_balanced)
        y_valid_pred_random_forest_hiper = model.predict(df_val.drop('Exited', axis=1))
        result = f1_score(df_val['Exited'], y_valid_pred_random_forest_hiper)
        if result > best_result:
            best_model = model
            best_result = result
            best_est = est
            best_depth = depth

print(f"Best model: {best_model}")
print(f"Best result: {best_result}")
print(f"Best n_estimators: {best_est}")
print(f"Best max_depth: {best_depth}")

print()
y_valid_pred_random_forest_hiper = best_model.predict(df_val.drop('Exited', axis=1))
print(f"F1 score: {f1_score(df_val['Exited'], y_valid_pred_random_forest_hiper)}")
print(f"Accuracy score: {accuracy_score(df_val['Exited'], y_valid_pred_random_forest_hiper)}")
print(f"Recall score: {recall_score(df_val['Exited'], y_valid_pred_random_forest_hiper)}")
print(f"ROC AUC Score: {roc_auc_score(df_val['Exited'], y_valid_pred_random_forest_hiper)}")

Best model: RandomForestClassifier(max_depth=9, n_estimators=81, random_state=12345)
Best result: 0.6214177978883861
Best n_estimators: 81
Best max_depth: 9

F1 score: 0.6214177978883861
Accuracy score: 0.8159824046920822
Recall score: 0.7518248175182481
ROC AUC Score: 0.7919674546306837


Podemos ver que seleccionando los mejores hiperparametros para este modelo, podemos mejorar todos los valores de nuestro modelo, pero el que mas nos interesa es el F1, el cual pasa de 0.596 a 0.621, lo cual es una mejora bastante buena.

## Conclusiones

Tomando en cuenta todo lo anterior, podemos decir que el mejor modelo para este caso es el de Random Forest Classifier, ya que nos da un F1 de 0.621, el cual es el mejor de todos los modelos que hemos probado. 

Para este proyecto en el que buscamos predecir si un cliente tiene la posibilidad de irse del banco, es muy importante observar no solo el valor de F1, si no todos los demas aqui expuesto para poder tener un modelo lo mas eficiente posible.

